# Which DNA Patterns Serve as Molecular Clocks?
In the DNA, there are regulatory sections, which is fragments that serves as a switch or conditional, rather than to be coded to protein. In this course, the section (regardless of regulatory or not) is called as `motif`.

Note: I think the course is quite inconsistent in naming things. Earlier it was called k-mers, substring, fragment, etc.

## 1.2 Motif Finding Is More Difficult Than You Think
Earlier on Week 2, we learn about frequent words with mismatches (remember function `frequent_words_with_mismatches`?). Now there's similar problem we approach. Below is the example mimics a transcription factor binding site hiding in the upstream regions of ten genes. Here we have k-mers of 15-mers. It's long enough to slow down the `frequent_words_with_mismatches`. We need better algorithm!

```sh
1 "atgaccgggatactgatAgAAgAAAGGttGGGggcgtacacattagataaacgtatgaagtacgttagactcggcgccgccg"
2 "acccctattttttgagcagatttagtgacctggaaaaaaaatttgagtacaaaacttttccgaatacAAtAAAAcGGcGGGa"
3 "tgagtatccctgggatgacttAAAAtAAtGGaGtGGtgctctcccgatttttgaatatgtaggatcattcgccagggtccga"
4 "gctgagaattggatgcAAAAAAAGGGattGtccacgcaatcgcgaaccaacgcggacccaaaggcaagaccgataaaggaga"
5 "tcccttttgcggtaatgtgccgggaggctggttacgtagggaagccctaacggacttaatAtAAtAAAGGaaGGGcttatag"
6 "gtcaatcatgttcttgtgaatggatttAAcAAtAAGGGctGGgaccgcttggcgcacccaaattcagtgtgggcgagcgcaa"
7 "cggttttggcccttgttagaggcccccgtAtAAAcAAGGaGGGccaattatgagagagctaatctatcgcgtgcgtgttcat"
8 "aacttgagttAAAAAAtAGGGaGccctggggcacatacaagaggagtcttccttatcagttaatgctgtatgacactatgta"
9 "ttggcccattggctaaaagcccaacttgacaaatggaagatagaatccttgcatActAAAAAGGaGcGGaccgaaagggaag"
10 "ctggtgagcaacgacagattcttacgtgcattagctcgcttccggggatctaatagcacgaagcttActAAAAAGGaGcGGa"
```

### Implanted Motif Problem (Motif Enumeration)
We'll try first with brute force approach. This one is motif enumeration. Such algorithms require little effort to design and are guaranteed to produce a correct solution, but they may take an enormous amount of time, and the number of candidates may be too large to check.

Given a collection of strings Dna and an integer d, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most d mismatches. For example, the implanted 15-mer in the strings above represents a (15,4)-motif.

In [ ]:
def hamming_distance(s1, s2):
    return sum([1 for i in range(len(s1)) if s1[i] != s2[i]])

def minimal_hamming_distance(pattern, dna):
    k = len(pattern)
    min_distance = k
    for i in range(len(dna) - k + 1):
        distance = min([hamming_distance(pattern, dna[i:i+k]) for i in range(len(dna) - k + 1)])
        if distance < min_distance:
            min_distance = distance
    return min_distance

def neighbors(pattern, d):
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return {'A', 'C', 'G', 'T'}
    neighborhood = set()
    suffix_neighbors = neighbors(pattern[1:], d)
    for text in suffix_neighbors:
        if hamming_distance(pattern[1:], text) < d:
            for x in ['A', 'C', 'G', 'T']:
                neighborhood.add(x + text)
        else:
            neighborhood.add(pattern[0] + text)
    return neighborhood

def pattern_count(text, pattern, d):
    count = 0
    for i in range(len(text) - len(pattern) + 1):
        if hamming_distance(text[i:i+len(pattern)], pattern) <= d:
            count += 1
    return count

def motif_enumeration(dna, k, d):
    patterns = set()
    joined_dna = ''.join(dna)
    for i in range(len(joined_dna) - k + 1):
        pattern = joined_dna[i:i+k]
        for neighbor in neighbors(pattern, d):
            if all([pattern_count(dna[i], neighbor, d) > 0 for i in range(len(dna))]):
                patterns.add(neighbor)
    return patterns

# Example usage
dna = ['ATTTGGC', 'TGCCTTA', 'CGGTATC', 'GAAAATT']
k = 3
d = 1
print(motif_enumeration(dna, k, d)) # Output: {'ATA', 'ATT', 'GTT', 'TTT'}

{'ATT', 'ATA', 'GTT', 'TTT'}


That was complicated isn't it? It took me several hours to figure out what the course wants and the pseudocode isn't consistent with the description.
Anyway, we should just forget that implementation and move on to better way of finding motif.

### Scoring Motifs
Scoring motifs is basically lining all dna fragment, and for each columns, counting the nucleotides that differs from the most common nucleotides in that row.

For example:
```sh
                    T   C   G   G 
                    T   C   G   a
Motifs              a   t   G   c
                    T   a   G   G
                    T   g   G   G
Score(Motifs)       1 + 3 + 0 + 2 = 6

                
                A:  1   1   0   1   
Count(Motifs)   C:  0   2   0   1
                G:  0   0   5   3
                T:  4   1   0   0

                A:  .2  .2  0   .2
Profile(Motifs) C:  0   .4  0   .2
                G:  0   0   1   .6
                T:  .8  .2  0   0

Cnsnsus(Motifs)     T   C   G   G
                
```

Every column of `Profile(Motifs)` corresponds to a probability distribution. So, we can use the `Profile(Motifs)` for counting the entropy.



For example, the 1st column of `Profile(Motifs)` entropy is:
```sh
    -(0.2 log2 0.2 + 0 log2 0 + 0 log2 0 + 0.8 log2 0.8) ≈ 0.72
```

The more conserved a nucleotide is, the smaller the entropy. See the case of column #3:
```sh
    -(0 log2 0 + 0 log2 0 + 1 log2 1 + 0 log2 0) ≈ 0
```

Note: Technically, log2(0) is undefined, but in the computation of entropy, we assume that 0 · log2(0) is equal to 0.